In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password
import json
import requests 
from pprint import pprint
import time
import pandas as pd
from sqlalchemy import create_engine 
from flask import jsonify

In [3]:
rds_connection_string = f"{username}:{password}@localhost:5432/park_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [4]:
engine.table_names()

/Users/lbl/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['park',
 'webcam_url',
 'images',
 'fire',
 'sci_name',
 'park_species',
 'activity',
 'park_activities',
 'park_fee',
 'fees']

In [7]:
#Pull table from DB and display first 5 rows
fire = pd.read_sql_query('select * from fire', con=engine)
fire.head()

fire_id                               park_id park_agency  \
0   238468  C0BF2A42-E353-4FAE-B4C4-AA0676B58100         NPS   
1   238469  C0BF2A42-E353-4FAE-B4C4-AA0676B58100         NPS   
2   238470  C0BF2A42-E353-4FAE-B4C4-AA0676B58100         NPS   
3   238471  C0BF2A42-E353-4FAE-B4C4-AA0676B58100         NPS   
4   238472  C0BF2A42-E353-4FAE-B4C4-AA0676B58100         NPS   

                         park_name   fire_name  fire_year cause_description  \
0  Denali National Park & Preserve  MOOSE CRK1       1993         Lightning   
1  Denali National Park & Preserve   TOLKAT RV       1993         Lightning   
2  Denali National Park & Preserve   KANTISHNA       1996     Miscellaneous   
3  Denali National Park & Preserve       DEPOT       1997           Smoking   
4  Denali National Park & Preserve       HOTEL       1997           Smoking   

   fire_size fire_size_class  latitude  longitude state  
0    11249.0               G   63.6828  -151.0856    AK  
1     3554.0               F   63.9162  -150.1190    AK  
2        1.0               B   63.5242  -150.9612    AK  
3        0.1               A   63.7359  -148.9140    AK  
4        0.1               A   63.7162  -148.9023    AK

In [54]:
#Group df by both 'fire_size_class' and 'fire_year' and show count 
# of 'fire_id'
fire_by_year = fire.groupby(['fire_size_class', 'fire_year']).count()
#Reset index
fire_by_year = fire_by_year.reset_index()
#Use Groupby method on 'fire_size_class' and use Agg method to make 
# list of: 'fire_year', 'fire_id'
fire_by_year = fire_by_year.groupby('fire_size_class').agg({'fire_year': list,
                                             'fire_id': list})
#Show df
fire_by_year

fire_year  \
fire_size_class                                                      
A                [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...   
B                [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...   
C                [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...   
D                [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...   
E                [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...   
F                [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...   
G                [1993, 1994, 1995, 1996, 1998, 1999, 2000, 200...   

                                                           fire_id  
fire_size_class                                                     
A                [674, 473, 738, 483, 553, 488, 437, 412, 451, ...  
B                [266, 214, 335, 222, 249, 195, 197, 219, 220, ...  
C                [58, 50, 73, 63, 92, 60, 66, 66, 56, 60, 62, 5...  
D                [19, 18, 23, 18, 22, 26, 12, 12, 17, 16, 10, 2...  
E                [11, 4, 19, 10, 19, 6, 14, 16, 8, 13, 9, 15, 4...  
F                [2, 7, 11, 5, 12, 4, 12, 8, 13, 15, 8, 14, 7, ...  
G                [3, 3, 2, 3, 3, 3, 6, 6, 4, 8, 2, 6, 8, 2, 6, ...